In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

from collections import Counter
import os
import sys

import matplotlib.pyplot as plt
import numpy as np

repo_root = os.path.join(os.getcwd(), '../code')
sys.path.append(repo_root)

import candidate_data
import imagenet
import mturk_data
import mturk_utils
import utils

In [2]:
imgnet = imagenet.ImageNetData()
cds = candidate_data.CandidateData(load_metadata_from_s3=False, exclude_blacklisted_candidates=False)

mturk = mturk_data.MTurkData(live=True, load_assignments=False)

Reading from local file /Users/ludwig/research/deep_learning/imagenet_2/data/cache/metadata/imagenet_metadata_2018-09-14_01-26-58_UTC.pickle ... done
Loaded 153637 unique candidates from 40 search result JSON file(s).
    /Users/ludwig/research/deep_learning/imagenet_2/data/search_results/...
        2018-07-31_flickr_search_result_vaishaal_class_1_153.json
        2018-08-20-16-10-18_becca.json
        2018-08-25-11-43-09_becca.json
        2018-08-27-22-53-45_becca.json
        2018-08-30-02-40-26_becca.json
        2018-08-30-18-46-35_becca.json
        2018-08-30-19-31-10_becca.json
        2018-09-04-17-03-01_becca.json
        2018-09-04-17-36-03_becca.json
        2018-09-05-16-16-14_becca.json
        ...
    There were 58222 duplicate occurences.
    Ignored 0 candidate entries because they are on the blacklist (blacklist size: 1203).
Loaded 5403 HITs from 28 hit data JSON file(s) in 0 seconds.
    /Users/ludwig/research/deep_learning/imagenet_2/data/mturk/hit_data_live/...
  

In [3]:
imgs = list(mturk.hits_of_image.keys())[0:1000:200]
for img in imgs:
    print('Hits of image {} : '.format(img))
    for hit in mturk.hits_of_image[img]:
        print('    {}'.format(hit['uuid']))

Hits of image 1582397371dd770737bfa9ea0c2813e229a6420c : 
    3c7987e5-737d-4f07-9905-35b1d76c181a
    ce3fc97a-4ce4-4f08-a66c-e2fa30adc391
    e3f3bbdc-f497-49a5-9c9e-707849be06b5
    f1b41450-481a-4548-8cee-0c49f8349c82
    44b3a590-9e6c-4fdb-8055-80133eeb2969
    9eae75a5-bc96-41f8-8bf8-1ef71635b3c7
Hits of image 479296a731192347d414ab39575740260ab6c0c6 : 
    b7a40c07-2de9-4b9a-83ec-971dcc0bc40e
Hits of image 0510d188d124aab763b6aac8c494b7a304a45863 : 
    fab3e8ab-8677-4731-b952-0961a818fe17
Hits of image 1fe994770481c890b10a8d79b251c82896d0448c : 
    83e6df72-2846-4cd6-b25b-076697c74115
    46b0c0d3-8b61-4ea2-ba28-35339d8ec962
    dc0efd86-1073-423e-976b-d0bacc81bf97
Hits of image ca5142fe9137a1e0bc9785473fa71428e87d2a63 : 
    d1864a6f-0f2e-44b4-965d-548d8fd87f3a


In [4]:
wnids = list(mturk.hits_for_wnid)[::200]
for wnid in wnids:
    print('Hits for wnid {} : '.format(wnid))
    for hit in mturk.hits_for_wnid[wnid]:
        print('    {}'.format(hit['uuid']))

Hits for wnid n01443537 : 
    3c7987e5-737d-4f07-9905-35b1d76c181a
    2d124821-9713-464c-bb16-b78d5c1c7519
    ce3fc97a-4ce4-4f08-a66c-e2fa30adc391
    05465731-2780-4f7c-b445-df576d79f8e9
    e3f3bbdc-f497-49a5-9c9e-707849be06b5
    1604e00c-1cf5-424a-a83c-d6c7f5bfdfe1
    f1b41450-481a-4548-8cee-0c49f8349c82
    c80c370c-bb9f-445f-960e-a36d4977e8eb
    44b3a590-9e6c-4fdb-8055-80133eeb2969
    755bcc0a-c3d7-4995-97f8-5f09a50f3ac5
    9eae75a5-bc96-41f8-8bf8-1ef71635b3c7
    865e35cf-2acd-4f53-8bc7-4745d464f55d
    204f81b1-bdf2-47c1-a9de-8c2f51132b45
    95172a30-733d-4b89-9bf3-144154428a69
    8d1b5bfd-d968-4775-bdc9-81f79dd7205f
Hits for wnid n03720891 : 
    d8993ad4-4227-4e1b-bb92-6051630b54e4
    b10b032a-fa5f-4f1a-a493-d4e6e9ae1c42
    fab8f485-2733-4d0a-abe0-66e478f1677d
    9fc9c093-123b-44ad-890d-1a08fb92da2d
    6b00097f-36db-4b48-8333-8311ccb91ff3
    4f3d0ee7-1d96-40f7-a208-b6dc869446a1
    b4b9fc24-e86b-42e5-a86d-90cc32e37603
    5a1e553f-7584-42bf-9725-90c881c3d015
   

In [5]:
hit_sizes = []
for hit in mturk.hits.values():
    hit_sizes.append(len(hit['images_all']))
c = Counter(hit_sizes)
for size, freq in c.most_common(100):
    print('There are {} HITs with size {}'.format(freq, size))

There are 5383 HITs with size 48
There are 20 HITs with size 25


In [6]:
print('Blacklisted HITs:')
for uuid in mturk.blacklisted_hits.keys():
    print(uuid)
    assert uuid not in mturk.hits

Blacklisted HITs:
70f3eff6-7eb9-47b8-a557-afd0befd6757
f2729bd9-d5c9-4bf7-aaeb-54c659ceecd8
a9ac30d5-4591-490a-9357-c023e6f9b7a2
238bff99-1bc7-4c52-931b-d57453d936b1


In [7]:
mturk = mturk_data.MTurkData(live=True,
                             load_assignments=False,
                             source_filenames_to_ignore=mturk_data.main_collection_filenames_to_ignore)

Loaded 5375 HITs from 21 hit data JSON file(s) in 0 seconds.
    /Users/ludwig/research/deep_learning/imagenet_2/data/mturk/hit_data_live/...
        vaishaal_hits_submitted_2018-08-25-10:18:38-PDT.json
        vaishaal_hits_submitted_2018-08-28-01:56:41-PDT.json
        vaishaal_hits_submitted_2018-08-29-18:55:37-PDT.json
        ...
    Ignored 7 filenames.
    Ignored 4 HITs from the blacklist (blacklist size 4)


In [8]:
hit_sizes = []
for hit in mturk.hits.values():
    hit_sizes.append(len(hit['images_all']))
c = Counter(hit_sizes)
for size, freq in c.most_common(100):
    print('There are {} HITs with size {}'.format(freq, size))

There are 5375 HITs with size 48
